In [1]:
import pennylane as qml
from pennylane import numpy as np
import jax
import qutip as qtp
from scipy.optimize import minimize

In [2]:
coeffs = [0.5]*3
ops = [qml.PauliZ(0) @ qml.PauliZ(1), qml.PauliZ(1) @ qml.PauliZ(2), qml.PauliZ(0) @ qml.PauliZ(2)]
isingHam = qml.Hamiltonian(coeffs, ops) # native Hamiltonian

In [3]:
n = 3 # total number of qubits
d = 2**n # dimension of composite system
H = isingHam # native Hamiltonian
tau = 1 # characteristic time

rho = np.zeros((d, d), dtype=np.complex128, requires_grad=False)
val = 0.25
rho[0, 0] = val
rho[1, 1] = val
rho[2, 2] = val
rho[3, 3] = val
rho = np.array(rho, requires_grad=False) # initial density matrix

wireList = list(range(n))
thetas = np.array(np.random.randn((d**2-1), requires_grad = True)) # random initial parameters for optimization

## CF1

In [4]:
devRho = qml.device("default.mixed", wires=3)
@qml.qnode(devRho)
def processRho(n, H, tau, thetas, rho):
    '''
    Time evolves rho using scrambled Hamiltonian by characteristic time, then traces out of environment.
    Inputs:
     - n: total number of qubits
     - H: native Hamiltonian to be scrambled
     - tau: characteristic time
     - thetas: parameters to scramble H
     - rho: initial density matrix
    
    Outputs:
     - rhoTau: time evolved, then traced out density matrix
    '''
    qml.QubitDensityMatrix(rho, wires=wireList)
    qml.adjoint(qml.SpecialUnitary(thetas, wires=wireList))
    qml.ApproxTimeEvolution(H, tau, 100)
    qml.SpecialUnitary(thetas, wires=wireList)
    return qml.density_matrix([0])

#print(qml.draw(circuit)(n, isingHam, tau, thetas, rho))

In [5]:
@qml.qnode(devRho)
def purity(thetas):
    '''
    Initializes a density matrix and takes its purity.
    '''
    qml.QubitDensityMatrix(processRho(n, H, tau, thetas, rho), wires=0)
    return qml.purity(0)

def cost(thetas):
    '''
    Computes linear entropy from purity. 
    '''
    return 1 - purity(thetas)

In [16]:
opt = minimize(cost, thetas)
cost(opt.x)

2.6441293599077653e-11

## Old code

In [18]:
devRho = qml.device("default.mixed", wires=3)
@qml.qnode(devRho)
def initRho(n):
    qml.QubitDensityMatrix(rho, wires=wireList)
    return qml.state()

In [19]:
@qml.qnode(devRho)
def TE(H, tau, rho):
    qml.QubitDensityMatrix(rho, wires=wireList)
    qml.adjoint(qml.SpecialUnitary(thetas, wires=wireList))
    qml.ApproxTimeEvolution(H, tau, 100)
    qml.SpecialUnitary(thetas, wires=wireList)
    return qml.density_matrix([0])
    
#print(qml.draw(TE)(isingHam, tau, rho))

In [24]:
@qml.qnode(devRho)
def score(rhoTau):
    qml.QubitDensityMatrix(rhoTau, wires=0)
    return qml.purity(0)

In [21]:
rho = initRho(n)
rhoTau = TE(isingHam, tau, rho)
score = score(rhoTau)

In [22]:
score

0.5726129954782994

In [ ]:
def scram(n, H, thetas):
    '''
    Scrambles Hamiltonian with GGMMs parameterized by thetas.
    '''
    U = qml.SpecialUnitary(thetas, wires=list(range(n)))
    Udag = qml.adjoint(U) 
    return qml.prod(U, H, Udag)

def constructTEO(H, thetas, wireList):
    U = qml.SpecialUnitary(thetas, wires = wireList)
    return qml.prod(U, H, qml.adjoint(U))